### Projet PACTE 4A-IIIA: Data Science Formula 1 World  Championship
#### Sujet: Quels sont les facteurs déterminants pour qu’un pilote ou une équipe atteigne la première place dans le championnat de Formule 1 ?

In [3]:
import pandas as pd

#### 1. Chargement des données

In [5]:
circuits = pd.read_csv("./dataset/circuits.csv")
constructor_results = pd.read_csv("./dataset/constructor_results.csv")
constructor_standings = pd.read_csv("./dataset/constructor_standings.csv")
constructors = pd.read_csv("./dataset/constructors.csv")
driver_standings = pd.read_csv("./dataset/driver_standings.csv")
drivers = pd.read_csv("./dataset/drivers.csv")
lap_times = pd.read_csv("./dataset/lap_times.csv")
pit_stops = pd.read_csv("./dataset/pit_stops.csv")
qualifying = pd.read_csv("./dataset/qualifying.csv")
races = pd.read_csv("./dataset/races.csv")
results = pd.read_csv("./dataset/results.csv")
seasons = pd.read_csv("./dataset/seasons.csv")
sprint_results = pd.read_csv("./dataset/sprint_results.csv")
status = pd.read_csv("./dataset/status.csv")

##### Verification de chargement

In [7]:
# Vérification des datasets importés
datasets = {
    "circuits": circuits,
    "constructor_results": constructor_results,
    "constructor_standings": constructor_standings,
    "constructors": constructors,
    "driver_standings": driver_standings,
    "drivers": drivers,
    "lap_times": lap_times,
    "pit_stops": pit_stops,
    "qualifying": qualifying,
    "races": races,
    "results": results,
    "seasons": seasons,
    "sprint_results": sprint_results,
    "status": status
}

for name, df in datasets.items():
    print(f"{name}: {df.shape[0]} lignes, {df.shape[1]} colonnes")

circuits: 77 lignes, 9 colonnes
constructor_results: 12505 lignes, 5 colonnes
constructor_standings: 13271 lignes, 7 colonnes
constructors: 212 lignes, 5 colonnes
driver_standings: 34595 lignes, 7 colonnes
drivers: 859 lignes, 9 colonnes
lap_times: 575029 lignes, 6 colonnes
pit_stops: 10990 lignes, 7 colonnes
qualifying: 10254 lignes, 9 colonnes
races: 1125 lignes, 18 colonnes
results: 26519 lignes, 18 colonnes
seasons: 75 lignes, 2 colonnes
sprint_results: 300 lignes, 16 colonnes
status: 139 lignes, 2 colonnes


#### 2. Préparation des données

#### 2.1. Exploration initiale des données
Cette étape permet d’avoir une vue d’ensemble des datasets (colonnes, types de données, valeurs manquantes).

In [9]:
# Afficher les premières lignes et résumés des datasets
for name, df in datasets.items():
    print(f"Dataset: {name}")
    print(df.info())
    print(df.head(), "\n")


Dataset: circuits
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   circuitId   77 non-null     int64  
 1   circuitRef  77 non-null     object 
 2   name        77 non-null     object 
 3   location    77 non-null     object 
 4   country     77 non-null     object 
 5   lat         77 non-null     float64
 6   lng         77 non-null     float64
 7   alt         77 non-null     int64  
 8   url         77 non-null     object 
dtypes: float64(2), int64(2), object(5)
memory usage: 5.5+ KB
None
   circuitId   circuitRef                            name      location  \
0          1  albert_park  Albert Park Grand Prix Circuit     Melbourne   
1          2       sepang    Sepang International Circuit  Kuala Lumpur   
2          3      bahrain   Bahrain International Circuit        Sakhir   
3          4    catalunya  Circuit de Barcelona-Catalunya     

##### 2.2. Traitement des valeurs manquantes
Les valeurs manquantes peuvent fausser les résultats. On identifie ces valeurs et choisit une stratégie : suppression, imputation, ou autre.

In [11]:
# Vérification des valeurs manquantes
for name, df in datasets.items():
    print(f"{name} - Valeurs manquantes:\n{df.isnull().sum()}\n")

circuits - Valeurs manquantes:
circuitId     0
circuitRef    0
name          0
location      0
country       0
lat           0
lng           0
alt           0
url           0
dtype: int64

constructor_results - Valeurs manquantes:
constructorResultsId    0
raceId                  0
constructorId           0
points                  0
status                  0
dtype: int64

constructor_standings - Valeurs manquantes:
constructorStandingsId    0
raceId                    0
constructorId             0
points                    0
position                  0
positionText              0
wins                      0
dtype: int64

constructors - Valeurs manquantes:
constructorId     0
constructorRef    0
name              0
nationality       0
url               0
dtype: int64

driver_standings - Valeurs manquantes:
driverStandingsId    0
raceId               0
driverId             0
points               0
position             0
positionText         0
wins                 0
dtype: int64

drivers 

Traitement des valeurs manquantes, nettoyage et conversions

In [13]:
# Nettoyage des données
for name, df in datasets.items():
    print(f"Nettoyage du dataset: {name}")

    # Vérifier et afficher les valeurs manquantes avant nettoyage
    print(f"{name} - Valeurs manquantes avant nettoyage:\n{df.isnull().sum()}\n")
    
    # Traitement des colonnes numériques (remplacement des valeurs manquantes par la moyenne)
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    df[numeric_cols] = df[numeric_cols].apply(lambda col: col.fillna(col.mean()))
    
    # Traitement des colonnes catégorielles (remplacement par la modalité la plus fréquente)
    categorical_cols = df.select_dtypes(include=['object']).columns
    df[categorical_cols] = df[categorical_cols].apply(lambda col: col.fillna(col.mode()[0] if not col.mode().empty else 'NA'))
    
    # Suppression des doublons
    df.drop_duplicates(inplace=True)
    
    # Réinitialisation de l'index
    df.reset_index(drop=True, inplace=True)
    
    # Vérifier et afficher les valeurs manquantes après nettoyage
    print(f"{name} - Valeurs manquantes après nettoyage:\n{df.isnull().sum()}\n")
    print(f"{name} - Taille après nettoyage: {df.shape}\n")

Nettoyage du dataset: circuits
circuits - Valeurs manquantes avant nettoyage:
circuitId     0
circuitRef    0
name          0
location      0
country       0
lat           0
lng           0
alt           0
url           0
dtype: int64

circuits - Valeurs manquantes après nettoyage:
circuitId     0
circuitRef    0
name          0
location      0
country       0
lat           0
lng           0
alt           0
url           0
dtype: int64

circuits - Taille après nettoyage: (77, 9)

Nettoyage du dataset: constructor_results
constructor_results - Valeurs manquantes avant nettoyage:
constructorResultsId    0
raceId                  0
constructorId           0
points                  0
status                  0
dtype: int64

constructor_results - Valeurs manquantes après nettoyage:
constructorResultsId    0
raceId                  0
constructorId           0
points                  0
status                  0
dtype: int64

constructor_results - Taille après nettoyage: (12505, 5)

Nettoyage d

In [19]:
# pour sauvegarder les datasets nettoyés si nécessaire
for name, df in datasets.items():
    df.to_csv(f"./dataset/cleaned_{name}.csv", index=False)

# Vérification du nettoyage

In [22]:
# Vérification du nettoyage
for name, df in datasets.items():
    print(f"Vérification du dataset: {name}")

    # Vérifier qu'il n'y a plus de valeurs manquantes
    missing_values = df.isnull().sum().sum()
    if missing_values == 0:
        print(f"✔ {name} : Pas de valeurs manquantes.")
    else:
        print(f"✘ {name} : Encore {missing_values} valeurs manquantes.\n{df.isnull().sum()}")
    
    # Vérifier les doublons
    duplicates = df.duplicated().sum()
    if duplicates == 0:
        print(f"✔ {name} : Pas de doublons.")
    else:
        print(f"✘ {name} : Encore {duplicates} doublons.")
    
    # Vérifier les dimensions
    print(f"{name} : {df.shape[0]} lignes, {df.shape[1]} colonnes.\n")


Vérification du dataset: circuits
✔ circuits : Pas de valeurs manquantes.
✔ circuits : Pas de doublons.
circuits : 77 lignes, 9 colonnes.

Vérification du dataset: constructor_results
✔ constructor_results : Pas de valeurs manquantes.
✔ constructor_results : Pas de doublons.
constructor_results : 12505 lignes, 5 colonnes.

Vérification du dataset: constructor_standings
✔ constructor_standings : Pas de valeurs manquantes.
✔ constructor_standings : Pas de doublons.
constructor_standings : 13271 lignes, 7 colonnes.

Vérification du dataset: constructors
✔ constructors : Pas de valeurs manquantes.
✔ constructors : Pas de doublons.
constructors : 212 lignes, 5 colonnes.

Vérification du dataset: driver_standings
✔ driver_standings : Pas de valeurs manquantes.
✔ driver_standings : Pas de doublons.
driver_standings : 34595 lignes, 7 colonnes.

Vérification du dataset: drivers
✔ drivers : Pas de valeurs manquantes.
✔ drivers : Pas de doublons.
drivers : 859 lignes, 9 colonnes.

Vérification du

In [24]:
import pandas as pd
import os

# Liste des noms de datasets à vérifier
dataset_names = [
    "circuits", "constructor_results", "constructor_standings",
    "constructors", "driver_standings", "drivers", "lap_times",
    "pit_stops", "qualifying", "races", "results", "seasons",
    "sprint_results", "status"
]

# Dictionnaire pour stocker les DataFrames nettoyés
cleaned_datasets = {}

# Charger les fichiers nettoyés
for name in dataset_names:
    file_path = f"./dataset/cleaned_{name}.csv"
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        cleaned_datasets[name] = df
    else:
        print(f"✘ Fichier manquant : {file_path}")

# Vérification des fichiers nettoyés
for name, df in cleaned_datasets.items():
    print(f"Vérification du fichier nettoyé: {name}")

    # Vérifier qu'il n'y a plus de valeurs manquantes
    missing_values = df.isnull().sum().sum()
    if missing_values == 0:
        print(f"✔ {name} : Pas de valeurs manquantes.")
    else:
        print(f"✘ {name} : Encore {missing_values} valeurs manquantes.\n{df.isnull().sum()}")

    # Vérifier les doublons
    duplicates = df.duplicated().sum()
    if duplicates == 0:
        print(f"✔ {name} : Pas de doublons.")
    else:
        print(f"✘ {name} : Encore {duplicates} doublons.")

    # Vérifier les dimensions
    print(f"{name} : {df.shape[0]} lignes, {df.shape[1]} colonnes.\n")


Vérification du fichier nettoyé: circuits
✔ circuits : Pas de valeurs manquantes.
✔ circuits : Pas de doublons.
circuits : 77 lignes, 9 colonnes.

Vérification du fichier nettoyé: constructor_results
✔ constructor_results : Pas de valeurs manquantes.
✔ constructor_results : Pas de doublons.
constructor_results : 12505 lignes, 5 colonnes.

Vérification du fichier nettoyé: constructor_standings
✔ constructor_standings : Pas de valeurs manquantes.
✔ constructor_standings : Pas de doublons.
constructor_standings : 13271 lignes, 7 colonnes.

Vérification du fichier nettoyé: constructors
✔ constructors : Pas de valeurs manquantes.
✔ constructors : Pas de doublons.
constructors : 212 lignes, 5 colonnes.

Vérification du fichier nettoyé: driver_standings
✔ driver_standings : Pas de valeurs manquantes.
✔ driver_standings : Pas de doublons.
driver_standings : 34595 lignes, 7 colonnes.

Vérification du fichier nettoyé: drivers
✔ drivers : Pas de valeurs manquantes.
✔ drivers : Pas de doublons.
dr

#### 3. Analyse descriptive des données

##### 3.1. Statistiques descriptives
Fournir des statistiques (moyenne, médiane, écart-type, etc.) pour comprendre la distribution des variables.